In [1]:
!pip install numba

#from numba import cuda
#device = cuda.get_current_device()
#device.reset()


#PYTORCH_CUDA_ALLOC_CONF=expandable_segments = True


In [2]:
!pip install datasets
!pip install transformers[torch]
! pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [3]:
from datasets import load_dataset
import json
import os
import glob



#get train data file
!wget -q -O train.txt "https://docs.google.com/uc?export=download&id=1cQBB074e3-xffKNuDgUok5M_u7iuykYZ"

#get train corrected data file
!wget -q -O test.txt 'https://docs.google.com/uc?export=download&id=1P0IJRSSwxgAMPJP98bHnJNgzHqG-PfRl'

#get val data file
!wget -q -O val.txt 'https://docs.google.com/uc?export=download&id=18AvCV0nwvxSUA8U102enCqzdsLHsf04E'

#splitting text data and putting into json format
def formatdata(filename):
    with open((filename+".txt"), "r", encoding = 'utf-8') as file:
        data = []
        for line in file:
            error_sentence, correct_sentence = line.strip().split('\t')
            data.append({'error_sentence': error_sentence, 'correct_sentence': correct_sentence})

        indexed_data = []
        for index, pair in enumerate(data):
            indexed_pair = {'index': index, **pair}
            indexed_data.append(indexed_pair)

    json_data = json.dumps(indexed_data, ensure_ascii=False, indent=4)

    # Write JSON data
    with open((filename+".json"), "w", encoding = 'utf-8') as json_file:
        json_file.write(json_data)


formatdata("train")
formatdata("test")
formatdata("val")

#loading dataset
raw_datasets = load_dataset("json", data_files={'train': 'train.json', 'test': 'test.json', 'val':'val.json'})


raw_datasets = raw_datasets.filter(lambda x: x["correct_sentence"] is not None)
raw_datasets = raw_datasets.filter(lambda x: x["error_sentence"] is not None)
data_sample = raw_datasets["train"].shuffle(seed=63).select(range(1000))
# Peek at data
data_sample[:3]

raw_train_dataset = raw_datasets["train"]
#need to index like in tutorial? labels??
raw_train_dataset[0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

{'correct_sentence': '사람들이 정년 연장은 고령화 사회의 해결 방법이라고 생각했다.',
 'index': 0,
 'error_sentence': '사람들이 정년을 연장은 고령화 사회의 해결 방법이라고 생각했다.'}

In [4]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="ko_KR", tgt_lang="ko_KR")


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [5]:
def preprocess_function(examples):
    inputs = examples["error_sentence"]
    targets = examples["correct_sentence"]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=128, truncation=True
    )
    return model_inputs


tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

tokenized_datasets

Map:   0%|          | 0/19898 [00:00<?, ? examples/s]

Map:   0%|          | 0/4265 [00:00<?, ? examples/s]

Map:   0%|          | 0/4264 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 19898
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4265
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4264
    })
})

In [6]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()
batch["decoder_input_ids"]

tensor([[     2, 250014,  14238,    138,   2680, 147886,  25006,  12812,   8280,
           7084,      6, 151327,   7063,   1963,   5837,   2166,  74534,  12624,
          25006,  12812,    469,  53941,  82017,   3156,   6571,   2461,   1020,
          92234,  92462,      5],
        [     2, 250014, 188425,   3184,  90546,    713,      6,  20427,  97854,
          34682,  11817,   2461,   1020,   2901,  76516, 133299,  29044,     32,
              2,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1]])

In [12]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"outputdir_me",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    push_to_hub=False,
)

In [13]:
from nltk.translate.gleu_score import sentence_gleu as gleu_score
from sklearn.metrics import precision_recall_fscore_support
!pip install evaluate
import evaluate
!pip install sacrebleu

gleu_metric = evaluate.load("google_bleu")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

bleu_metric = evaluate.load("sacrebleu")
metrics = [gleu_metric, precision_metric, recall_metric, f1_metric]


def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]


    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    #for p, l in zip(decoded_preds, decoded_labels):
        #print(f"pred: {p}, label: {l}")

    gleu = gleu_score(decoded_labels, decoded_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average='weighted')
    #gleu = gleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #precision = precision_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #recall = recall_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #f1=f1_metric.compute(predictions=decoded_preds, references=decoded_labels)



    return {
      "bleu": bleu["score"],
      "gleu": gleu,
      "f1": f1,
      "precision": precision,
      "recall": recall
      }


In [14]:
#make trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

trainer.evaluate(max_length=128)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.5020730495452881,
 'eval_bleu': 62.20816603870951,
 'eval_gleu': 5.865102639296188e-05,
 'eval_f1': 0.16369606003752346,
 'eval_precision': 0.16381332082551595,
 'eval_recall': 0.16369606003752346,
 'eval_runtime': 435.5204,
 'eval_samples_per_second': 9.791,
 'eval_steps_per_second': 1.224}

In [ ]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 

In [ ]:
trainer.evaluate(max_length=128)

In [ ]:
#next(model.parameters()).is_cuda

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

In [ ]:
import torch

def finetuned_generate(text, model, tokenizer):
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  corrected_ids = model.generate(torch.tensor([input_ids]),
                                  max_length=128,
                                  eos_token_id=1, num_beams=4,
                                  early_stopping=True, repetition_penalty=2.0)

  output_text = tokenizer.decode(corrected_ids.squeeze().tolist(), skip_special_tokens=True)
  return output_text

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to('cpu')
text = '한국어이어렵다고 생각하고 잇다.'
output_text=finetuned_generate(text, model, tokenizer)
print(output_text)
